In [1]:
"""
まずは
"""
import pandas as pd

dir = "../data/"

df = pd.read_csv(dir + "windy_value/date/value_bottom_val_date.csv", header=None)
print(df)

#dataディレクトリの下のディレクトリ
windy_value = ["date","max_windy_speed","swell","swell_spacing","time_hour","wave_height","wind_speed"]

#data/*/ディレクトリの中の*_top_*.csvファイル
value_top_data = ["value_top_val_date.csv","value_top_val_max_wind_speed.csv","value_top_val_swell.csv",
                  "value_top_val_swell_spacing.csv","value_top_val_time_hour.csv","value_top_val_wave_height.csv",
                 "value_top_val_wind_speed.csv"]

             0           1           2           3           4           5  \
0   2020-08-13  2020-08-14  2020-08-15  2020-08-16  2020-08-17  2020-08-18   
1   2020-09-01  2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06   
2   2020-09-02  2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07   
3   2020-09-03  2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08   
4   2020-09-04  2020-09-05  2020-09-06  2020-09-07  2020-09-08  2020-09-09   
..         ...         ...         ...         ...         ...         ...   
63  2020-12-13  2020-12-14  2020-12-15  2020-12-16  2020-12-17  2020-12-18   
64  2020-12-14  2020-12-15  2020-12-16  2020-12-17  2020-12-18  2020-12-19   
65  2020-12-15  2020-12-16  2020-12-17  2020-12-18  2020-12-19  2020-12-20   
66  2020-12-16  2020-12-17  2020-12-18  2020-12-19  2020-12-20  2020-12-21   
67  2020-12-17  2020-12-18  2020-12-19  2020-12-20  2020-12-21  2020-12-22   

             6           7           8           9  
0   2020-0

In [2]:
    eng_name_dic = {
        "taketomi_route" : ["isigaki_dep", "taketomi_dep"],
        "kurosima_route" : ["isigaki_dep", "kurosima_dep"],
        "kohama_route" : ["isigaki_dep", "tkohama_dep"],
        "iriomote_uehara_route" : ["isigaki_dep", "tuehara_dep"],
        "hatoma_route" : ["isigaki_dep", "hatoma_dep"],
        "iriomote_ohara_route" : ["isigaki_dep", "ohara_dep"],
       "hateruma_route" : ["isigaki_dep", "hateruma_dep"]
                                               } 

In [3]:
import pathlib
import glob
import re
import pprint
import os

csv_file = [os.path.basename(p) for p in glob.glob(dir + '/route/taketomi_route/isigaki_dep/*', recursive=True)
     if os.path.isfile(p) and re.search(r'2020-[0|1][8|9|10|11|12]-.*',p)]
csv_file = sorted(csv_file)

#print(csv_file)

print(re.findall('(.*).csv', csv_file[0]))
date_data = re.findall('(.*).csv', csv_file[0])
# ['2020-08-01']

date_in = df[df[0].isin([date_data])]
print(date_in)

df_bool = df[0] == date_data[0]
print(df_bool.values.sum())
if df_bool.values.sum() == 0:
    #for文continu
    pass
else:
    #日付がある場合なら処理(windy_valueがあるということ)
    
    pass



['2020-08-02']
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Index: []
0


In [15]:
#日付の計算を行うための
import pandas.tseries.offsets as offsets

dir_label = dir + 'route/taketomi_route/isigaki_dep/' +csv_file[12]
print(dir_label)
def label_df(dir_label) :
    df = pd.read_csv(dir_label, header=None)
    df[2]= pd.to_datetime(df[2])
    print(df)
    for row in df.itertuples():
        print(row[3]+offsets.Day(-15))
        print(type(row[3]))
        #print(row.point)
        #if row[0] < 
    
label_df(dir_label)

../data/route/taketomi_route/isigaki_dep/2020-08-14.csv
        0  1          2
0   07:30  0 2020-08-14
1   08:30  0 2020-08-14
2   09:30  0 2020-08-14
3   10:30  0 2020-08-14
4   11:30  0 2020-08-14
5   13:00  0 2020-08-14
6   14:00  0 2020-08-14
7   15:00  0 2020-08-14
8   16:00  0 2020-08-14
9   17:00  0 2020-08-14
10  18:00  0 2020-08-14
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2020-07-30 00:00: